In [88]:
import numpy as np # linear algebra
import pandas as pd # data processing, csv file I/O (e.g. pd.read_csv)
import os # deal with os primitives
import time # time-related functions
import matplotlib.pyplot as plt # plots
import pickle # object serialization
from collections import Counter # dict subclass for counting hashable items
from tqdm import tqdm # iterable object
import random # random values generator
import seaborn as sns # prettier plots
import torch # main package for PyTorch
import torch.utils.data as data_utils # access data sets, including pre-processing, loading, and splitting
from torch.utils.data import random_split # randomly split a dataset 
import torch.optim as optim # optimization algorithms
import torch.nn as nn # build neural network (layers, activations, loss functions)
import torch.nn.functional as F # functions used to build neural network
from torchsummary import summary # print the summary of a neural network model
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts # scheduler used to adjust the learning rate
from torch.utils.tensorboard.writer import SummaryWriter # nn log writer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import torch

# set the random seed for reproducibility
torch.manual_seed(1111)
torch.cuda.manual_seed(1111)
np.random.seed(1111)
random.seed(1111)

In [89]:
import pandas as pd

# Load the train set
df_train = pd.DataFrame(pd.read_pickle("df_train.pkl"))

# Load the test set
df_test = pd.DataFrame(pd.read_pickle("df_test.pkl"))


In [90]:
df_train.head()

,graph,label_time,label_space
0,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13)",bfs,dfs
1,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",dfs,dfs
2,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",best_first_search,astar
3,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11)",bfs,best_first_search
4,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",best_first_search,astar


In [91]:
df_train

,graph,label_time,label_space
0,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13)",bfs,dfs
1,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",dfs,dfs
2,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",best_first_search,astar
3,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11)",bfs,best_first_search
4,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",best_first_search,astar
...,...,...,...
1795,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",dfs,best_first_search
1796,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",best_first_search,astar
1797,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",best_first_search,best_first_search
1798,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",best_first_search,randomized_shortest_path


In [92]:
for _, row in df_train.iterrows():
    graph = row["graph"]
    label = row["label_time"]

    # Check for node features
    node_features_available = set()
    for node in graph.nodes:
        node_features_available.update(graph.nodes[node].keys())

    print("Node Features Available:", node_features_available)

    # Check for edge features
    edge_features_available = set()
    for edge in graph.edges:
        edge_features_available.update(graph.edges[edge].keys())

    print("Edge Features Available:", edge_features_available)

    break

Node Features Available: {'coord2d'}
Edge Features Available: {'weight'}


In [93]:
#Choosing the number or epochs and the learning rate
num_epochs = 1000
batch_size = 256
mini_batch_size = 64

In [94]:
df_train["label_time"].unique()

array(['bfs', 'dfs', 'best_first_search', 'bidirectional_search',
       'dijkstra', 'randomized_shortest_path', 'astar'], dtype=object)

In [116]:
import pandas as pd
import networkx as nx
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data
from sklearn.model_selection import train_test_split

# Convert NetworkX graph to PyTorch Geometric Data
def convert_nx_to_pyg_data(nx_graph, label):
    # Extract node features from NetworkX graph
    node_features = torch.tensor([list(nx_graph.nodes[node]['coord2d']) for node in nx_graph.nodes], dtype=torch.float32)

    # Extract edge indices from NetworkX graph as pairs
    edge_index = torch.tensor([[edge[0], edge[1]] for edge in nx_graph.edges], dtype=torch.long).t().contiguous()

    # If your graph has edge features, extract them similarly
    edge_features = torch.tensor([nx_graph.edges[edge]['weight'] for edge in nx_graph.edges], dtype=torch.float32).view(-1, 1)

    # Create a PyTorch Geometric Data object
    data = Data(x=node_features, edge_index=edge_index, edge_attr=edge_features, y=label)
    return data

# Convert each row in df_train to PyTorch Geometric Data
label_mapping = {
    'bfs': 0,
    'dfs': 1,
    'best_first_search': 2,
    'bidirectional_search': 3,
    'dijkstra': 4,
    'randomized_shortest_path': 5,
    'astar': 6
}

data_list = []
for _, row in df_train.iterrows():
    graph = row["graph"]
    label_str = row["label_time"]
    # Convert the string label to a numerical value using the mapping
    label_num = label_mapping[label_str]
    # Assuming graph is a NetworkX graph
    pyg_data = convert_nx_to_pyg_data(graph, label_num)
    data_list.append(pyg_data)

# Split the data into train and validation sets
train_data, val_data = train_test_split(data_list, test_size=0.1, random_state=42)

# Assuming you have a test dataset in a similar format
test_data_list = []
for _, row in df_test.iterrows():
    graph = row["graph"]
    label_str = row["label_time"]
    label_num = label_mapping[label_str]
    pyg_data = convert_nx_to_pyg_data(graph, label_num)
    test_data_list.append(pyg_data)

# Create DataLoader for the test set
test_dataloader = DataLoader(test_data_list, batch_size=batch_size, shuffle=False)


In [117]:
print(data_list[0].x)
print(data_list[0].edge_index)
print(data_list[0].edge_attr)

tensor([[11.5583,  0.7371],
        [10.7485,  1.0607],
        [ 6.1089,  5.3043],
        [ 0.1407, 13.0577],
        [ 2.8523,  3.9811],
        [ 7.5872,  1.0717],
        [ 0.8846,  3.4299],
        [ 8.4719,  0.3661],
        [ 0.9402,  0.2484],
        [13.2872, 11.3556],
        [ 7.2208, 12.8095],
        [ 8.0238,  7.3646],
        [ 8.3400, 12.2960],
        [10.4877,  9.4897]])
tensor([[ 0,  0,  1,  1,  4,  5,  6],
        [ 4, 13, 12, 13,  6,  8,  8]])
tensor([[0.1765],
        [0.9303],
        [0.3943],
        [0.3801],
        [0.1946],
        [0.3115],
        [0.2308]])


In [118]:
import torch
from torch_geometric.data import Data

# Accessing the features of the first data instance
first_data_instance = train_data[3]
node_features = first_data_instance.x
graph_connectivity = first_data_instance.edge_index
edge_features = first_data_instance.edge_attr
target_label = first_data_instance.y

# Perform further analysis or operations as needed
# For example, you can print the shapes of the tensors
print("Node Features Shape:", node_features.shape)
print("Graph Connectivity Shape:", graph_connectivity.shape)
print("Edge Features Shape:", edge_features.shape)
print("Target Label:", target_label)


Node Features Shape: torch.Size([44, 2])
Graph Connectivity Shape: torch.Size([2, 211])
Edge Features Shape: torch.Size([211, 1])
Target Label: 2


In [119]:
#Choosing the number or epochs and the learning rate
num_epochs = 1000
batch_size = 64
mini_batch_size = 32

# Create DataLoader for training and validation sets
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

In [120]:
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda')
        print("[i] USING CUDA")
    else:
        device = torch.device('cpu') # don't have GPU 
        print("[i] USING CPU")
    return device

device = get_device() #setting up the DL device

[i] USING CPU


In [121]:
num_classes = 7
input_dim = 2

In [122]:
import time
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, precision_score, recall_score
# define training function

# implement early stopping for training function
# from https://stackoverflow.com/questions/71998978/early-stopping-in-pytorch


class EarlyStopper:
    def __init__(self, patience=1, min_delta=0.0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = np.inf

    def early_stop(self, validation_loss):
        if validation_loss <= self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                print(f"[i] Validation Loss Increased - Early Stop!")
                print(
                    f"--- {validation_loss} > {self.min_validation_loss + self.min_delta} ---")
                return True
        return False


def train(net, train_loader, validation_loader, num_epochs, mini_batch_size, optimizer, lr_scheduler, criterion, earlystop_patience=0, earlystop_min_delta=1e-6, name=""):
    # Save the loss into a dataframe
    losses = pd.DataFrame(index=list(range(num_epochs)), columns=[
                          'running_loss', 'train_loss', 'valid_loss'])
    min_validation_loss = np.inf

    # Use a summary writer to check loss in real time
    current_time = time.strftime("%Y%m%d_%H%M%S")
    writer = SummaryWriter(
        f'runs/tensorboard/{current_time}_{(net.__class__.__name__).lower()}_{name}')

    # Set early stopping parameters
    # from https://stackoverflow.com/questions/71998978/early-stopping-in-pytorch
    early_stopping = EarlyStopper(
        patience=earlystop_patience, min_delta=earlystop_min_delta)

    start_time_epoch = time.time()

    net.train()
    net.to(device)  # Move the model to the specified device

    for epoch in range(num_epochs):  # Looping over the dataset

        running_loss = 0.0
        validation_loss = 0.0
        train_loss = 0.0

        net.train()  # Set the model to training mode

        for i, data in enumerate(train_loader):
            start_time_mini_batch = time.time()
            inputs = data.to(device)

            optimizer.zero_grad()  # Setting the parameter gradients to zero
            outputs = net(inputs)  # Forward pass

            target_one_hot = F.one_hot(data.y, num_classes=num_classes)  # Adjust the number of classes accordingly

            #print(target_one_hot, outputs)

            loss = criterion(outputs, target_one_hot.float())
            loss.backward()
            optimizer.step()  # Optimization step

            running_loss += loss.item()  # Updating the running loss
            train_loss += loss.item()

            if i % mini_batch_size == mini_batch_size - 1:  # Printing the running loss
                print(f"[epoch: {epoch + 1}, mini-batch: {i + 1}, time-taken: {round(time.time() - start_time_mini_batch, 3)} sec] loss: {round(running_loss / mini_batch_size, 6)} ")

                # write on the summary writer
                writer.add_scalar(
                    'Loss/Running', running_loss / mini_batch_size, i)

                running_loss = 0.0
                start_time_mini_batch = time.time()


        net.eval().to(device)

        # Inside the validation loop
        with torch.no_grad():
            net.eval()  # Set the model to evaluation mode
            all_labels = []
            all_outputs = []

            for i, data in enumerate(validation_loader):
                inputs = data.to(device)

                optimizer.zero_grad()  # Setting the parameter gradients to zero
                outputs = net(inputs)  # Forward pass

                target_one_hot = F.one_hot(data.y, num_classes=num_classes)  # Adjust the number of classes accordingly

                loss = criterion(outputs, target_one_hot.float())

                validation_loss += loss.item()  # Check the loss

                # Use argmax to get the index of the predicted class
                predicted_class = torch.argmax(outputs, dim=1)

                # Append predictions and labels for accuracy calculation
                all_outputs.extend(predicted_class.cpu().numpy())
                all_labels.extend(data.y.cpu().numpy())

            # Convert lists to numpy arrays for easier computation
            all_labels = np.array(all_labels)
            all_outputs = np.array(all_outputs)

            val_accuracy = accuracy_score(all_labels, all_outputs)
            val_precision = precision_score(all_labels, all_outputs, average='weighted', zero_division=1.0)
            val_recall = recall_score(all_labels, all_outputs, average='weighted', zero_division=1.0)
            val_f1 = f1_score(all_labels, all_outputs, average='weighted', zero_division=1.0)

            # Print or log the accuracy and validation loss
            print(f'+++ [\033[1mepoch: {epoch + 1}\033[0m, validation - \033[91maccuracy: {val_accuracy:.5f}\033[0m, \033[93mprecision: {val_precision:.5f}\033[0m, \033[94mrecall: {val_recall:.5f}\033[0m, \033[95mf1-score: {val_f1:.5f}\033[0m] +++')

        # Switch back to training mode for the next epoch
        net.train().to(device)

        print('+++ [epoch: %d, training loss: %.5f, validation loss: %.5f] +++' %
              (epoch + 1,
               train_loss / len(train_loader),
               validation_loss / len(validation_loader)))

        print(
            f"--- time-taken for epoch {epoch+1}: {round(time.time() - start_time_epoch, 3)} seconds ---")
        start_time_epoch = time.time()

        # Saving the loss
        losses.at[epoch, 'running_loss'] = running_loss
        losses.at[epoch, 'train_loss'] = train_loss
        losses.at[epoch, 'valid_loss'] = validation_loss

        # Write on the summary writer
        writer.add_scalar('Loss/Train', train_loss / len(train_loader), epoch)
        writer.add_scalar('Loss/Validation', validation_loss /
                          len(validation_loader), epoch)

        # Update the learning rate
        if lr_scheduler.__class__.__name__ == "CosineAnnealingWarmRestarts" and lr_scheduler is not None:
            print(f"\033[90m--- current LR: {round(lr_scheduler.get_last_lr()[0], 9)} ---\033[0m")
            lr_scheduler.step()  # step scheduler learning rate

        if min_validation_loss > (validation_loss / len(validation_loader)):
            print(f'\033[92m+++ [validation loss decreased ({min_validation_loss:.9f} -> {(validation_loss / len(validation_loader)):.9f}), saving the model ...] +++\033[0m')
            min_validation_loss = validation_loss / len(validation_loader)

            # Check if the directory exists, and if not, create it
            save_dir = f'./runs/models/{(net.__class__.__name__).lower()}'
            os.makedirs(save_dir, exist_ok=True)

            # Save State Dict
            torch.save(net.state_dict(), f'{save_dir}/{(net.__class__.__name__).lower()}_{name}_saved_model.pth')

        # Check if early stopping criteria is fulfilled
        if early_stopping.early_stop(validation_loss):
            break

    pickle.dump(losses, open(
        f'./runs/models/{(net.__class__.__name__).lower()}/{(net.__class__.__name__).lower()}_{name}_loss.pkl', 'wb'))
    writer.close()
    print(f"[i] Finished Training")

In [123]:
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool, BatchNorm
from torch_geometric.data import DataLoader, Data

class ComplexGNN(nn.Module):
    def __init__(self, input_dim, output_dim, init_fn):
        super(ComplexGNN, self).__init__()
        hidden_dim1 = 1024
        hidden_dim2 = 512
        hidden_dim3 = 256
        hidden_dim4 = 128
        hidden_dim5 = 64

        self.conv1 = GCNConv(input_dim, hidden_dim1)
        self.bn1 = BatchNorm(hidden_dim1)
        self.conv2 = GCNConv(hidden_dim1, hidden_dim2)
        self.bn2 = BatchNorm(hidden_dim2)
        self.conv3 = GCNConv(hidden_dim2, hidden_dim3)
        self.bn3 = BatchNorm(hidden_dim3)
        self.conv4 = GCNConv(hidden_dim3, hidden_dim4)
        self.bn4 = BatchNorm(hidden_dim4)
        self.conv5 = GCNConv(hidden_dim4, hidden_dim5)
        self.bn5 = BatchNorm(hidden_dim5)

        # Fully connected layers
        fc_hidden_dim1 = 64
        fc_hidden_dim2 = 32
        fc_hidden_dim3 = 16 

        self.fc1 = nn.Linear(hidden_dim5, fc_hidden_dim1)
        self.fc2 = nn.Linear(fc_hidden_dim1, fc_hidden_dim2)
        self.fc3 = nn.Linear(fc_hidden_dim2, fc_hidden_dim3)
        self.fc4 = nn.Linear(fc_hidden_dim3, output_dim)

        self.init_fn = init_fn
        self.apply(self.init_weights)

    def init_weights(self, m):
        if isinstance(m, nn.Linear):
            self.init_fn(m.weight)
            m.bias.data.fill_(0.01)

    def forward(self, data):
        # Ensure edge_index has the correct shape
        edge_index = data.edge_index.view(2, -1)

        # Extract relevant data
        x, batch, edge_attr = data.x, data.batch, data.edge_attr

        # Apply graph convolutional layers with batch normalization
        x = F.elu(self.bn1(self.conv1(x, edge_index)))
        x = F.elu(self.bn2(self.conv2(x, edge_index)))
        x = F.elu(self.bn3(self.conv3(x, edge_index)))
        x = F.elu(self.bn4(self.conv4(x, edge_index)))
        x = F.elu(self.bn5(self.conv5(x, edge_index)))

        # Global pooling to obtain a representation for each graph
        x = global_mean_pool(x, batch)

        # Fully connected layers
        x = F.elu(self.fc1(x))
        x = F.elu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)

        return x

In [124]:
model = ComplexGNN(input_dim=2, output_dim=num_classes, init_fn=torch.nn.init.xavier_normal_).to(device)  # Adjust output_dim based on your task
train_flag = False # Dont run if False
name = "gnn_complex"

if train_flag:
    print(f"[i] Training the network {model.__class__.__name__} ...")

    # Learning rate
    learning_rate = 3e-3
    
    # Define your criterion (e.g., CrossEntropyLoss for multiclass classification)
    criterion = nn.CrossEntropyLoss().to(device)
    
    # Define your optimizer (e.g., Adam)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Cosine Annealing with Restarts (CWR) scheduler.
    # This scheduler is designed to automatically adjust the learning rate according to a cosine wave, and can be used to adjust the learning rate as the model converges.
    scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=round((1/10)*batch_size), T_mult=1, eta_min=0)

    # Train the model
    train(model, train_dataloader, val_dataloader, num_epochs, mini_batch_size, optimizer, scheduler, criterion, name=name)
else:  # load the model
    print(f"[i] Loading the network {model.__class__.__name__} ...")
    # Loading existing models (with saved weights)
    model.load_state_dict(torch.load(f'./runs/models/{(model.__class__.__name__).lower()}/{(model.__class__.__name__).lower()}_{name}_saved_model.pth', map_location=device))  # using saved data if present
    model.eval()


[i] Training the network ComplexGNN ...
+++ [epoch: 1, validation - accuracy: 0.63889, precision: 0.66890, recall: 0.63889, f1-score: 0.50889] +++
+++ [epoch: 1, training loss: 1.57004, validation loss: 1.42927] +++
--- time-taken for epoch 1: 13.0 seconds ---
--- current LR: 0.003 ---
+++ [validation loss decreased (inf -> 1.429269711), saving the model ...] +++
+++ [epoch: 2, validation - accuracy: 0.63889, precision: 0.76929, recall: 0.63889, f1-score: 0.49812] +++
+++ [epoch: 2, training loss: 1.02655, validation loss: 1.50627] +++
--- time-taken for epoch 2: 13.585 seconds ---
--- current LR: 0.002799038 ---
[i] Validation Loss Increased - Early Stop!
--- 4.518815755844116 > 4.287810133529663 ---
[i] Finished Training


In [125]:
# Set the model to evaluation mode
model.eval()

# Initialize lists to store true labels and predicted labels
all_labels = []
all_outputs = []

# Loop through the test dataset
with torch.no_grad():
    for i, data in enumerate(test_dataloader):
        inputs = data.to(device)
        outputs = model(inputs)

        target_one_hot = F.one_hot(data.y, num_classes=num_classes)
        loss = criterion(outputs, target_one_hot.float())

        predicted_class = torch.argmax(outputs, dim=1)

        # Append predictions and labels for accuracy calculation
        all_outputs.extend(predicted_class.cpu().numpy())
        all_labels.extend(data.y.cpu().numpy())

# Convert lists to numpy arrays for easier computation
all_labels = np.array(all_labels)
all_outputs = np.array(all_outputs)

print(all_outputs)
val_accuracy = accuracy_score(all_labels, all_outputs)
val_precision = precision_score(all_labels, all_outputs, average='weighted', zero_division=1.0)
val_recall = recall_score(all_labels, all_outputs, average='weighted', zero_division=1.0)
val_f1 = f1_score(all_labels, all_outputs, average='weighted', zero_division=1.0)

print(f'+++ [\033[0m, validation - \033[91maccuracy: {val_accuracy:.5f}\033[0m, \033[93mprecision: {val_precision:.5f}\033[0m, \033[94mrecall: {val_recall:.5f}\033[0m, \033[95mf1-score: {val_f1:.5f}\033[0m] +++')

# Compute confusion matrix
conf_matrix = confusion_matrix(all_labels, all_outputs, normalize="pred")
print(conf_matrix)


[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
+++ [, validation - accuracy: 0.65000, precision: 0.77250, recall: 0.65000, f1-score: 0.51212] +++
[[0.   0.   0.04 0.   0.   0.  ]
 [0.   0.   0.14 0.   0.   0.  ]
 [0.   0.   0.65 0.   0.   0.  ]
 [0.   0.   0.13 0.   0.   0.  ]
 [0.   0.   0.02 0.   0.   0.  ]
 [0.   0.   0.02 0.   0.   0.  ]]
